# Rate depedent 1D SLIDE model

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

### Model implementation (single material point)

In [ ]:
def get_bond_slip(s_arr, t_arr, tau_pi_bar, K, gamma, E_b, S, c, r, m, sigma_n, eta):

    # arrays to store the values
    tau_arr = np.zeros_like(s_arr)
    alpha_arr = np.zeros_like(s_arr)
    z_arr = np.zeros_like(s_arr)
    w_arr = np.zeros_like(s_arr)
    s_pi_arr = np.zeros_like(s_arr)
    s_vp_arr = np.zeros_like(s_arr)
    tau_infty_arr = np.zeros_like(s_arr)
    tau_vp_arr = np.zeros_like(s_arr)
    f = np.zeros_like(s_arr)
    s_i1, w_i1, s_pi_i1, alpha_i1, z_i1, tau_i1, s_vp_i1, tau_infty, tau_vp = (
        0, 0, 0, 0, 0, 0, 0, 0, 0
    )

    dt_arr = t_arr[1:] - t_arr[:-1]
    ds_dt_arr = (s_arr[1:] - s_arr[:-1]) / dt_arr

    for i, ds_dt in enumerate(ds_dt_arr):
        s_i, w_i, s_pi_i, s_vp_i, alpha_i, z_i, tau_i = (
            s_arr[i], w_arr[i], s_pi_arr[i], s_vp_arr[i],
            alpha_arr[i], z_arr[i], tau_arr[i]
        )
        dt = dt_arr[i]
        i1 = i+1
        ds_i1 = ds_dt * dt
        s_i1 = s_i + ds_i1
        tau_trial = (1 - w_i) * E_b * (s_i1 - s_vp_i)
        
        # Threshold
        f_pi_i1 = np.fabs(tau_trial/(1. - w_i) - gamma * alpha_i) - tau_pi_bar - K * z_i + m * sigma_n

        if f_pi_i1 > 1e-6:
            # Return mapping
            delta_lambda = f_pi_i1 / (E_b / (1. - w_i) + gamma + K)
            # update all the state variables
            s_vp_i1 = s_vp_i + delta_lambda * np.sign(tau_trial/(1. - w_i) - gamma * alpha_i) / (1 - w_i)
            Y_i1 = 0.5 * E_b * (s_i1 - s_vp_i) ** 2
            w_i1 = w_i + ((1. - w_i) ** c) * (delta_lambda * (Y_i1 / S) ** r)  * (1 - sigma_n / (0.5 * tau_pi_bar)) 
            alpha_infty = alpha_i +  delta_lambda * np.sign(tau_trial/(1. - w_i) - gamma * alpha_i)
            z_infty = z_i + delta_lambda
            tau_infty = E_b * (1. - w_i1) * (s_i1 - s_vp_i1)
                        
            # perform viscoplastic regularization
            relax_t = eta / (E_b + K + gamma)
            
            dt_tau = dt / relax_t
            tau_vp = tau_i + E_b * ds_i1
            tau_i1 = (tau_vp + dt_tau * tau_infty)/(1 + dt_tau)
            z_i1 = (z_i + dt_tau * z_infty)/(1 + dt_tau) 
            alpha_i1 = (alpha_i + dt_tau * alpha_infty)/(1 + dt_tau) 
            s_vp_i1 = s_i1 - tau_i1/E_b

        i1 = i+1
        w_arr[i1] = w_i
        alpha_arr[i1] = alpha_i1
        z_arr[i1] = z_i1
        s_pi_arr[i1] = s_pi_i1
        s_vp_arr[i1] = s_vp_i1
        tau_infty_arr[i1] = tau_infty
        tau_vp_arr[i1] = tau_vp
        tau_arr[i1] = tau_i1

    return s_arr, tau_arr, w_arr, s_pi_arr, s_vp_arr, tau_infty_arr, tau_vp_arr


### Apply loading

In [ ]:
# Define the loading values

number_of_cycles = 2
#maximum_slip = 0.5
maximum_slip = 50
maximum_time = 10

number_of_increments_1 = 100
number_of_increments_2 = 100
number_of_increments_3 = 100


# Construction of the loading history
s_levels = np.linspace(0, maximum_slip, 2 * number_of_cycles )
s_levels[0] = 0
s_levels.reshape(-1, 2)[:, 0] *= -1
s_history = s_levels.flatten()


# slip array as input
s_arr_1 = np.hstack([np.linspace(s_history[i], s_history[i + 1], number_of_increments_1)
                     for i in range(len(s_levels) - 1)])
t_arr_1 = np.linspace(0, maximum_time, len(s_arr_1))
s_arr_2 = np.hstack([np.linspace(s_history[i], s_history[i + 1], number_of_increments_2)
                     for i in range(len(s_levels) - 1)])
t_arr_2 = np.linspace(0, maximum_time, len(s_arr_2))
s_arr_3 = np.hstack([np.linspace(s_history[i], s_history[i + 1], number_of_increments_3)
                     for i in range(len(s_levels) - 1)])
t_arr_3 = np.linspace(0, maximum_time, len(s_arr_3))



In [ ]:

# material parameters
tau_pi_bar=1
K=0
gamma=0
E_b=10
S=100000.0005
c=1
r=1.0
m=0.0
sigma_n=0
eta = 1

dt_1 = 1./number_of_increments_1
dt_2 = 1./number_of_increments_2
dt_3 = 1./number_of_increments_3

eta_1, eta_2, eta_3 = 0.1, 1, 10
s_arr_1,  tau_arr_1, w_arr_1, s_pi_arr_1, s_vis_arr_1, tau_infty_1, tau_vp_1 = get_bond_slip(
    s_arr_1, t_arr_1, tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S, c=c, r=r, 
    m=m, sigma_n=sigma_n, eta=eta_1)

s_arr_2,  tau_arr_2, w_arr_2, s_pi_arr_2, s_vis_arr_2, tau_infty_2, tau_vp_2 = get_bond_slip(
    s_arr_2, t_arr_2, tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S, c=c, r=r, 
    m=m, sigma_n=sigma_n, eta=eta_2)

s_arr_3,  tau_arr_3, w_arr_3, s_pi_arr_3, s_vis_arr_3, tau_infty_2, tau_vp_3 = get_bond_slip(
    s_arr_3, t_arr_3, tau_pi_bar=tau_pi_bar, K=K, gamma=gamma, E_b=E_b, S=S, c=c, r=r, 
    m=m, sigma_n=sigma_n, eta=eta_3)

### Plotting

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax1,  ax2), (ax3,  ax4)) = plt.subplots(2,2, tight_layout=True, figsize=(7,7))
fig.canvas.header_visible=False

ax1.plot(s_arr_1, tau_arr_1, 'k', linewidth=1, label="eta = {}".format(eta_1))
ax1.plot(s_arr_2, tau_arr_2, 'r', linewidth=1, label="eta = {}".format(eta_2))
ax1.plot(s_arr_3, tau_arr_3, 'g', linewidth=1, label="eta = {}".format(eta_3))
ax1.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax1.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('stress-slip')
plt.xlabel('Slip (mm)')
plt.ylabel('Stress (MPa)')
ax1.legend(loc=4)

ax2.plot(s_arr_1, s_vis_arr_1, 'k', linewidth=1, label="eta = {}".format(eta_1))
ax2.plot(s_arr_2, s_vis_arr_2, 'r', linewidth=1, label="eta = {}".format(eta_2))
ax2.plot(s_arr_3, s_vis_arr_3, 'g', linewidth=1, label="eta = {}".format(eta_3))
ax2.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax2.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Evolution of viscoplastic strain')
plt.xlabel('Slip(mm)')
plt.ylabel('viscoplastic strain')
ax2.legend()

ax3.plot(s_arr_1, w_arr_1, 'k', linewidth=1, label="eta = {}".format(eta_1))
ax3.plot(s_arr_2, w_arr_2, 'r', linewidth=1, label="eta = {}".format(eta_2))
ax3.plot(s_arr_3, w_arr_3, 'g', linewidth=1, label="eta = {}".format(eta_3))
ax3.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax3.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Damage evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('damage')
ax3.legend()

ax4.plot(t_arr_1, s_arr_1, 'k', linewidth=1, label="eta = {}".format(eta_1))
ax4.plot(t_arr_2, s_arr_2, 'r', linewidth=1, label="eta = {}".format(eta_2))
ax4.plot(t_arr_3, s_arr_3, 'g', linewidth=1, label="eta = {}".format(eta_3))
ax4.axhline(y=0, color='k', linewidth=1, alpha=0.5)
ax4.axvline(x=0, color='k', linewidth=1, alpha=0.5)
plt.title('Damage evolution')
plt.xlabel('Slip(mm)')
plt.ylabel('damage')
ax4.legend()

## Relaxation test

Reproduce the analytical solution of step loading with the decay of stress towards the time-independent solution

In [ ]:
# material parameters
tau_pi_bar=1
K=0
gamma=0.5
E_b=10
S=100000.0005
c=1
r=1.0
m=0.0
sigma_n=0
eta = 1

In [ ]:
import sympy as sp
from cymbol import Cymbol
s_1 = Cymbol('s_1', codename='s_1')
t_1 = Cymbol('t_1', codename='t_1')
t = Cymbol('t')
fn_s_t = sp.Piecewise(
    (t * s_1, t < t_1),
    (s_1, True)
)
get_s_t = sp.lambdify((t, s_1, t_1), fn_s_t)

In [ ]:
s_1 = 0.12
t_1 = 0.01
t_max = 1
n_t = 1000
t_arr = np.linspace(0, t_max, n_t+1)
s_arr = get_s_t(t_arr, s_1, t_1)

In [ ]:
s_arr_1, tau_arr_1, w_arr_1, s_pi_arr_1, s_vis_arr_1, tau_infty_1, tau_vp_1 = get_bond_slip(s_arr, t_arr, tau_pi_bar=tau_pi_bar, K=K,
              gamma=gamma, E_b=E_b, S=S, c=c, r=r, m=m, sigma_n=sigma_n, eta=eta)

In [ ]:
tau_infty_1.shape

In [ ]:
fig, (ax, ax_s_time) = plt.subplots(1,2)
ax.plot(t_arr, tau_arr_1)
ax_s_time.plot(t_arr, tau_infty_1)
# ax_s_time = ax.twinx()
# ax_s_time.plot(t_arr, s_arr, color='green')